In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext
import pandas as pd
import configparser
import matplotlib
import glob
import os
import boto3
from pyspark.sql.functions import udf
from pyspark.sql import types
from pyspark.sql.functions import expr

In [2]:
config = configparser.ConfigParser()
config.read_file(open(os.path.join(os.getcwd(),'dl.cfg')))
print(os.path.join(os.getcwd(),'dl.cfg'))

/home/workspace/dl.cfg


In [3]:
config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read_file(open(os.path.join(os.getcwd(),'dl.cfg')))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

In [4]:
from pyspark import SparkConf

In [5]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

In [6]:
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=config['AWS']['AWS_ACCESS_KEY_ID'],
                       aws_secret_access_key= config['AWS']['AWS_SECRET_ACCESS_KEY']
                   )


In [7]:
#s3 = boto3.resource('s3')
os.getcwd()

'/home/workspace'

In [8]:
LogDF = spark.read.json("s3a://udacity-dend/log_data/*/*/*.json")

In [9]:
SongDF = spark.read.json("s3a://udacity-dend/song_data/A/A/*/*.json",mode='DROPMALFORMED')

In [10]:
pd.set_option('max_colwidth', 200)
LogDF.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20100101 Firefox/31.0,12


In [11]:
LogDF.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [12]:
pd.set_option('max_colwidth', 200)
Log_nextsong =LogDF.where(LogDF['page'] =='NextSong')
Log_nextsong.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4""",61
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",80


In [13]:
Log_nextsong_ts=Log_nextsong.withColumn('timeStamp',expr("cast(cast(ts/1000 as int) as timestamp)"))
Log_nextsong_ts.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timeStamp
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26,2018-11-15 00:30:26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26,2018-11-15 00:41:21
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26,2018-11-15 00:45:41
3,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4""",61,2018-11-15 03:44:09
4,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",80,2018-11-15 05:48:55


In [20]:
Log_nextsong_ts.createOrReplaceTempView('LogView')

In [ ]:
 # create timestamp column from original timestamp column
    #get_timestamp = udf()

    
    # create datetime column from original timestamp column
   # get_datetime = udf()


In [21]:
dimUsers = spark.sql("""SELECT DISTINCT cast(userId as int),firstName,
                lastName, gender,level
                FROM LogView
             """)
dimUsers.limit(5).toPandas()

,userId,firstName,lastName,gender,level
0,26,Ryan,Smith,M,free
1,49,Chloe,Cuevas,F,free
2,29,Jacqueline,Lynch,F,free
3,15,Lily,Koch,F,paid
4,8,Kaylee,Summers,F,free


In [22]:
dimTime = spark.sql("""SELECT DISTINCT timeStamp AS start_time,hour(timeStamp) AS hour, \
            day(timeStamp) AS day,month(timeStamp) AS month,year(timeStamp) AS year,
            weekday(timeStamp) AS weekday,weekofyear(timeStamp) AS week FROM LogView
            """)
dimTime.limit(5).toPandas()

,start_time,hour,day,month,year,weekday,week
0,2018-11-15 14:57:44,14,15,11,2018,3,46
1,2018-11-15 16:02:17,16,15,11,2018,3,46
2,2018-11-15 18:19:34,18,15,11,2018,3,46
3,2018-11-21 03:57:19,3,21,11,2018,2,47
4,2018-11-21 13:40:39,13,21,11,2018,2,47


In [19]:
SongDF.createOrReplaceTempView('SongView')

In [23]:
dimArtist = spark.sql("""SELECT DISTINCT artist_id,artist_name,
                artist_location,artist_latitude,artist_longitude
                FROM SongView
                """)
dimArtist.limit(5).toPandas()           

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARB57BN1187B9B5EAF,36 Crazyfists,"Anchorage, AK",61.21756,-149.85776
1,ARMI4NV1187B99D55D,Man Man,"Philadelphia, PA",39.95227,-75.16237
2,ARKCTSM11F4C83C839,Igor Stravinsky,,NaN,NaN
3,AR8YYNB1187B9A4BB3,Assemblage 23,,NaN,NaN
4,ARDDQKN1187FB50651,Rednex,,NaN,NaN


In [38]:
#dimTimes3://charles-sink2/dimUsers/
#Log_nextsong_ts
dimTime.write.mode("overwrite").partitionBy("year","month").parquet("s3a://charles-sink/dimTIme")

In [37]:
dimUsers.write.mode("overwrite").parquet("s3a://charles-sink/dimUsers/") 